In [56]:
import pandas as pd
import numpy as np
import csv
import sqlite3
import re
import pickle

from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
%matplotlib inline

In [57]:
filename_csv = '/Users/xzhou/github/project_files/lending_club/loan.csv'
date_to_parse = ['issue_d' , 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 'earliest_cr_line']

df=pd.read_csv(filename_csv, date_parser=date_to_parse)

/Users/xzhou/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
df.shape

(887379, 74)

In [59]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

In [ ]:
# filename_sql = '/Users/xzhou/github/project_files/lending_club/database.sqlite'
# # date_to_parse = ['issue_d' , 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 'earliest_cr_line']
# conn = sqlite3.connect(filename_sql)

# df = pd.read_sql_query('select * from loan', con=conn, parse_dates = date_to_parse)

In [ ]:
# df.columns

In [60]:
filter = ['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc', 'loan_status', 'purpose', 
       'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'application_type',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'annual_inc_joint', 'dti_joint', 'acc_now_delinq',
          'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl',
       'inq_last_12m']

In [61]:
df_smaller = df.filter(items=filter)

In [62]:
df_smaller.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc', 'loan_status', 'purpose', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'application_type',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'annual_inc_joint', 'dti_joint', 'acc_now_delinq',
       'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m',
       'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
       'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m'],
      dtype='object')

In [63]:
df_smaller.isnull().sum()

loan_amnt                           0
term                                0
int_rate                            0
installment                         0
grade                               0
emp_length                      44825
home_ownership                      0
annual_inc                          4
loan_status                         0
purpose                             0
dti                                 0
delinq_2yrs                        29
earliest_cr_line                   29
inq_last_6mths                     29
mths_since_last_delinq         454312
mths_since_last_record         750326
open_acc                           29
pub_rec                            29
revol_bal                           0
revol_util                        502
application_type                    0
collections_12_mths_ex_med        145
mths_since_last_major_derog    665676
policy_code                         0
annual_inc_joint               886868
dti_joint                      886870
acc_now_deli

In [64]:
# Drop the filds have 90%+ null values, 
# as filling in the nulls can skew the model
df_smaller.drop(['annual_inc_joint', 'dti_joint',  'open_acc_6m', 'open_il_6m',
       'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
       'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
        'inq_fi', 'total_cu_tl', 'inq_last_12m'],
              axis=1, inplace=True)

In [65]:
df_smaller.rename(index=str, columns={
    "dti": "payment_to_income_ratio_individual",
}, inplace=True)

In [66]:
df_smaller.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc', 'loan_status', 'purpose',
       'payment_to_income_ratio_individual', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'application_type',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'total_rev_hi_lim'],
      dtype='object')

In [67]:
df_smaller.head()

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,loan_status,purpose,...,revol_bal,revol_util,application_type,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,5000.0,36 months,10.65,162.87,B,10+ years,RENT,24000.0,Fully Paid,credit_card,...,13648.0,83.7,INDIVIDUAL,0.0,NaN,1.0,0.0,NaN,NaN,NaN
1,2500.0,60 months,15.27,59.83,C,< 1 year,RENT,30000.0,Charged Off,car,...,1687.0,9.4,INDIVIDUAL,0.0,NaN,1.0,0.0,NaN,NaN,NaN
2,2400.0,36 months,15.96,84.33,C,10+ years,RENT,12252.0,Fully Paid,small_business,...,2956.0,98.5,INDIVIDUAL,0.0,NaN,1.0,0.0,NaN,NaN,NaN
3,10000.0,36 months,13.49,339.31,C,10+ years,RENT,49200.0,Fully Paid,other,...,5598.0,21.0,INDIVIDUAL,0.0,NaN,1.0,0.0,NaN,NaN,NaN
4,3000.0,60 months,12.69,67.79,B,1 year,RENT,80000.0,Current,other,...,27783.0,53.9,INDIVIDUAL,0.0,NaN,1.0,0.0,NaN,NaN,NaN


In [68]:
df_smaller.info

<bound method DataFrame.info of         loan_amnt        term  int_rate  installment grade emp_length  \
0          5000.0   36 months     10.65       162.87     B  10+ years   
1          2500.0   60 months     15.27        59.83     C   < 1 year   
2          2400.0   36 months     15.96        84.33     C  10+ years   
3         10000.0   36 months     13.49       339.31     C  10+ years   
4          3000.0   60 months     12.69        67.79     B     1 year   
5          5000.0   36 months      7.90       156.46     A    3 years   
6          7000.0   60 months     15.96       170.08     C    8 years   
7          3000.0   36 months     18.64       109.43     E    9 years   
8          5600.0   60 months     21.28       152.39     F    4 years   
9          5375.0   60 months     12.69       121.45     B   < 1 year   
10         6500.0   60 months     14.65       153.45     C    5 years   
11        12000.0   36 months     12.69       402.54     B  10+ years   
12         9000.0  

In [69]:
df_clean = df_smaller.copy()

In [70]:
df_clean.isnull().sum()

loan_amnt                                  0
term                                       0
int_rate                                   0
installment                                0
grade                                      0
emp_length                             44825
home_ownership                             0
annual_inc                                 4
loan_status                                0
purpose                                    0
payment_to_income_ratio_individual         0
delinq_2yrs                               29
earliest_cr_line                          29
inq_last_6mths                            29
mths_since_last_delinq                454312
mths_since_last_record                750326
open_acc                                  29
pub_rec                                   29
revol_bal                                  0
revol_util                               502
application_type                           0
collections_12_mths_ex_med               145
mths_since

In [71]:
excluded = ['Issued',
                'Does not meet the credit policy. Status:Charged Off',
                'Does not meet the credit policy. Status:Fully Paid',
               ]
good = ['Current', 'Fully Paid']
bad = ['Charged Off', 'Default', 'In Grace Period', 'Late (16-30 days)', 'Late (31-120 days)']

In [72]:
def get_status (status):
    """
    Assign loans to either good or bad loans based on their status
    """
    try: 
        status=status.strip()
        if status in good:
            return "good"
        elif status in bad:
            return "bad"
        else: 
            return "excluded"
    except: 
        return "excluded"

In [73]:
df_clean['loan_status'] = df_clean['loan_status'].apply(lambda x: get_status(x))

In [74]:
df_clean = df_clean[df_clean['loan_status']!='excluded']

In [75]:
df_clean.groupby('loan_status').count()

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,purpose,payment_to_income_ratio_individual,...,revol_bal,revol_util,application_type,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
loan_status,,,,,,,,,,,,,,,,,,,,,
bad,66668,66668,66668,66668,66668,62973,66668,66668,66668,66668,...,66668,66607,66668,66662,15124,66668,66668,56417,56417,56417
good,809502,809502,809502,809502,809502,769037,809502,809502,809502,809502,...,809502,809104,809502,809452,204054,809502,809502,752226,752226,752226


In [76]:
df_clean.isnull().sum()

loan_amnt                                  0
term                                       0
int_rate                                   0
installment                                0
grade                                      0
emp_length                             44160
home_ownership                             0
annual_inc                                 0
loan_status                                0
purpose                                    0
payment_to_income_ratio_individual         0
delinq_2yrs                                0
earliest_cr_line                           0
inq_last_6mths                             0
mths_since_last_delinq                449097
mths_since_last_record                741573
open_acc                                   0
pub_rec                                    0
revol_bal                                  0
revol_util                               459
application_type                           0
collections_12_mths_ex_med                56
mths_since

In [77]:
df_clean.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc', 'loan_status', 'purpose',
       'payment_to_income_ratio_individual', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'application_type',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'total_rev_hi_lim'],
      dtype='object')

In [78]:
# Check null values for rest of records

df_clean.isnull().sum()

loan_amnt                                  0
term                                       0
int_rate                                   0
installment                                0
grade                                      0
emp_length                             44160
home_ownership                             0
annual_inc                                 0
loan_status                                0
purpose                                    0
payment_to_income_ratio_individual         0
delinq_2yrs                                0
earliest_cr_line                           0
inq_last_6mths                             0
mths_since_last_delinq                449097
mths_since_last_record                741573
open_acc                                   0
pub_rec                                    0
revol_bal                                  0
revol_util                               459
application_type                           0
collections_12_mths_ex_med                56
mths_since

In [79]:
# Fill the null value for 'mths_since_last_delinq' with zero as zero is default case for majority of loans

fill_with_zero = {
    'mths_since_last_delinq':0,
    'mths_since_last_record':0,
    'collections_12_mths_ex_med':0,
    'mths_since_last_major_derog':0,
    'tot_coll_amt':0
}
            
df_clean.fillna(fill_with_zero,inplace=True)

In [80]:

df_clean.isnull().sum()

loan_amnt                                 0
term                                      0
int_rate                                  0
installment                               0
grade                                     0
emp_length                            44160
home_ownership                            0
annual_inc                                0
loan_status                               0
purpose                                   0
payment_to_income_ratio_individual        0
delinq_2yrs                               0
earliest_cr_line                          0
inq_last_6mths                            0
mths_since_last_delinq                    0
mths_since_last_record                    0
open_acc                                  0
pub_rec                                   0
revol_bal                                 0
revol_util                              459
application_type                          0
collections_12_mths_ex_med                0
mths_since_last_major_derog     

In [81]:
# Fill the null value for 'revol util' with means

df_clean['revol_util'].fillna(
    value=df_clean['revol_util'].mean(), inplace=True)

In [82]:
# Fill the null value for 'tot_cur_bal' with means

df_clean['tot_cur_bal'].fillna(
    value=df_clean['tot_cur_bal'].mean(), inplace=True)

In [83]:
# Fill the null value for 'total_rev_hi_lim' with means

df_clean['total_rev_hi_lim'].fillna(
    value=df_clean['total_rev_hi_lim'].mean(), inplace=True)

In [84]:
df_clean.isnull().sum()

loan_amnt                                 0
term                                      0
int_rate                                  0
installment                               0
grade                                     0
emp_length                            44160
home_ownership                            0
annual_inc                                0
loan_status                               0
purpose                                   0
payment_to_income_ratio_individual        0
delinq_2yrs                               0
earliest_cr_line                          0
inq_last_6mths                            0
mths_since_last_delinq                    0
mths_since_last_record                    0
open_acc                                  0
pub_rec                                   0
revol_bal                                 0
revol_util                                0
application_type                          0
collections_12_mths_ex_med                0
mths_since_last_major_derog     

In [85]:
def get_employee_length(length):
    """
    This is to extract numerical value of emp_length
    For simplicity, approximation was used.
    <1 years will be treated as 1 year
    >10 years will be treated as 10 years
    """
    try:
        length = length.strip()
        year = re.findall(r'(\d+)', length)
        return int(year[0]) 
    except:
        return None

In [86]:
df_clean['emp_length'] = df_clean['emp_length'].apply(lambda x: get_employee_length(x))

In [87]:
df_clean['emp_length'].fillna(
    value=df_clean['emp_length'].mean(), inplace=True)

In [88]:
# All Null values have been appropriately handled

df_clean.isnull().sum()

loan_amnt                             0
term                                  0
int_rate                              0
installment                           0
grade                                 0
emp_length                            0
home_ownership                        0
annual_inc                            0
loan_status                           0
purpose                               0
payment_to_income_ratio_individual    0
delinq_2yrs                           0
earliest_cr_line                      0
inq_last_6mths                        0
mths_since_last_delinq                0
mths_since_last_record                0
open_acc                              0
pub_rec                               0
revol_bal                             0
revol_util                            0
application_type                      0
collections_12_mths_ex_med            0
mths_since_last_major_derog           0
policy_code                           0
acc_now_delinq                        0


In [89]:
# Break down terms of loan into 2 categories

df_clean = pd.get_dummies(df_clean, prefix=['term'],columns=['term'])

In [90]:
df_clean.columns

Index(['loan_amnt', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc', 'loan_status', 'purpose',
       'payment_to_income_ratio_individual', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'application_type',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'total_rev_hi_lim', 'term_ 36 months', 'term_ 60 months'],
      dtype='object')

In [91]:
# Break down grades into categories

df_clean = pd.get_dummies(df_clean, prefix=['grade'],columns=['grade'])

In [92]:
df_clean.columns

Index(['loan_amnt', 'int_rate', 'installment', 'emp_length', 'home_ownership',
       'annual_inc', 'loan_status', 'purpose',
       'payment_to_income_ratio_individual', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'application_type',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'total_rev_hi_lim', 'term_ 36 months', 'term_ 60 months', 'grade_A',
       'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G'],
      dtype='object')

In [93]:
df_clean = pd.get_dummies(df_clean, prefix=['home_ownership'], columns=['home_ownership'])

In [94]:
df_clean.columns

Index(['loan_amnt', 'int_rate', 'installment', 'emp_length', 'annual_inc',
       'loan_status', 'purpose', 'payment_to_income_ratio_individual',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'application_type',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'total_rev_hi_lim', 'term_ 36 months', 'term_ 60 months', 'grade_A',
       'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G',
       'home_ownership_ANY', 'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT'],
      dtype='object')

In [95]:
df_clean=pd.get_dummies(df_clean, prefix=['purpose'], columns=['purpose'])

In [96]:
df_clean.columns

Index(['loan_amnt', 'int_rate', 'installment', 'emp_length', 'annual_inc',
       'loan_status', 'payment_to_income_ratio_individual', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'application_type', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'acc_now_delinq',
       'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'term_ 36 months',
       'term_ 60 months', 'grade_A', 'grade_B', 'grade_C', 'grade_D',
       'grade_E', 'grade_F', 'grade_G', 'home_ownership_ANY',
       'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
      

In [97]:
def get_earliest_cr_line(string):
    """
    This is to calculate years of credit history.
    Since the data is from 2016, use 2016 minus the 
    starting year to estimate years of credit history
    
    """
    string = string.strip()
    year = re.findall(r'(\d+)', string)
    return (2016-int(year[0])) 

In [98]:
df_clean['earliest_cr_line'] = df_clean['earliest_cr_line'].apply(lambda x: get_earliest_cr_line(x))

In [99]:
def inverse(value):
    try:
        return 1/value
    except: 
        return 0

In [100]:
# Take inverse of months since last delinquishing date as this is is inversed related to default prediction

df_clean['inversed_mths_since_last_delinq']=df_clean['mths_since_last_delinq'].apply(lambda x: inverse(x))

In [101]:
# Take inverse of months since last record as this is is inversed related to default to default prediction

df_clean['inversed_mths_since_last_record']=df_clean['mths_since_last_record'].apply(lambda x: inverse(x))

In [102]:
# Take inverse of months since last record as this is is inversed related to default to default prediction

df_clean['inversed_mths_since_last_major_derog']=df_clean['mths_since_last_major_derog'].apply(lambda x: inverse(x))

In [103]:
df_clean.drop(
    ['mths_since_last_delinq', 'mths_since_last_record',
     'mths_since_last_major_derog'], 
    inplace=True, axis=1)

In [105]:
df_clean['income_to_debt_ratio']=df_clean['annual_inc']/df_clean['revol_bal']

In [106]:
# Drop infinite value
df_clean['income_to_debt_ratio'] = df_clean['income_to_debt_ratio'].replace(np.inf, np.nan)

In [107]:
df_clean = df_clean[df_clean['income_to_debt_ratio']>=0]

In [109]:
df_clean=pd.get_dummies(df_clean, prefix=['application_type'], columns=['application_type'])

In [110]:
df_clean.columns

Index(['loan_amnt', 'int_rate', 'installment', 'emp_length', 'annual_inc',
       'loan_status', 'payment_to_income_ratio_individual', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'term_ 36 months', 'term_ 60 months', 'grade_A', 'grade_B', 'grade_C',
       'grade_D', 'grade_E', 'grade_F', 'grade_G', 'home_ownership_ANY',
       'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding',

In [111]:
df_clean.describe(include='all')

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,payment_to_income_ratio_individual,delinq_2yrs,earliest_cr_line,inq_last_6mths,...,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,inversed_mths_since_last_delinq,inversed_mths_since_last_record,inversed_mths_since_last_major_derog,income_to_debt_ratio,application_type_INDIVIDUAL,application_type_JOINT
count,872926.000000,872926.000000,872926.000000,872926.000000,8.729260e+05,872926,872926.000000,872926.000000,872926.000000,872926.000000,...,872926.000000,872926.000000,872926.000000,872926.000000,872926.000000,872926.000000,872926.000000,872926.000000,872926.000000,872926.000000
unique,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,good,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,806601,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,14779.352803,13.252395,437.392570,6.101334,7.500838e+04,NaN,18.182628,0.314092,18.306563,0.684289,...,0.000635,0.011380,0.005272,0.002560,0.031165,0.003001,0.009687,24.496675,0.999488,0.000512
std,8427.927252,4.384235,243.959234,3.444948,6.469140e+04,NaN,17.290395,0.861302,7.432795,0.966162,...,0.025184,0.106069,0.072416,0.050535,0.081515,0.014310,0.037072,742.312695,0.022623,0.022623
min,500.000000,5.320000,16.080000,1.000000,0.000000e+00,NaN,0.000000,0.000000,4.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8000.000000,9.990000,261.880000,3.000000,4.500000e+04,NaN,11.940000,0.000000,13.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.186915,1.000000,0.000000
50%,13000.000000,12.990000,382.810000,6.098617,6.500000e+04,NaN,17.680000,0.000000,17.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.136547,1.000000,0.000000
75%,20000.000000,16.200000,573.060000,10.000000,9.000000e+04,NaN,23.960000,0.000000,22.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.007092,9.139470,1.000000,0.000000


In [54]:
df_clean.isnull().sum()

loan_amnt                             0
int_rate                              0
installment                           0
emp_length                            0
annual_inc                            0
loan_status                           0
payment_to_income_ratio_individual    0
delinq_2yrs                           0
earliest_cr_line                      0
inq_last_6mths                        0
mths_since_last_delinq                0
mths_since_last_record                0
open_acc                              0
pub_rec                               0
revol_bal                             0
revol_util                            0
collections_12_mths_ex_med            0
mths_since_last_major_derog           0
policy_code                           0
acc_now_delinq                        0
tot_coll_amt                          0
tot_cur_bal                           0
total_rev_hi_lim                      0
term_ 36 months                       0
term_ 60 months                       0


In [112]:
pkl_filename= '/Users/xzhou/github/project_files/lending_club/loan3.pkl'

with open(pkl_filename, 'wb') as pklfile:
    pickle.dump(df_clean, pklfile)